In [1]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 

In [2]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler
import logging
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
import statistics
import torch.optim.lr_scheduler as lr_scheduler
from scipy.special import softmax
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset


In [3]:
logging.getLogger('mne').setLevel(logging.WARNING)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import import_ipynb
#from Model import net
from Dataloader2 import EEGDataset

importing Jupyter notebook from Dataloader2.ipynb


In [4]:
def Normalize(raw):
    normals = []
    scaler = StandardScaler()
    for idx in range(len(raw)):
        normals.append(scaler.fit_transform(raw[idx]))

    return raw


In [5]:
def test():
    logits = torch.argmax(model(X_unseen.cuda()), axis =1)
    logits = F.one_hot(logits.squeeze(), num_classes=4).float()
    ground_truth = F.one_hot(y_unseen.cuda().squeeze(), num_classes=4).float()
    acc = torch.eq(logits, ground_truth).all(dim=1)
    acc = torch.count_nonzero(acc) / acc.numel()
    torch.cuda.empty_cache()
    print(acc)

In [6]:
class net(nn.Module):
    def __init__(self, T, C, input_size, hidden_size, num_layers, spatial_num, dropout, pool):
        super(net, self).__init__()
        
        self.T = T
        self.C = C
        self.spatial_num = spatial_num
        self.dropout = dropout
        self.pool = pool

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.cell_count = self.T // self.input_size

        self.fcn_in = (spatial_num * self.hidden_size)

        self._lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)

        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])

        self.cnn_block = nn.Sequential(nn.Conv2d(1, self.spatial_num, (self.C, 1)),
                                       nn.BatchNorm2d(self.spatial_num),
                                       nn.ELU(),



                                       nn.Dropout(self.dropout))

        
        self.fcn = nn.Sequential(nn.Linear(self.fcn_in, 128), 
                                 nn.ReLU(),
                                 nn.Linear(128, 16),
                                 nn.Dropout(self.dropout),
                                 nn.ReLU(),
                                 nn.Linear(16, 4))

        #self.fcn = nn.Linear(self.fcn_in, 4)
        self.results = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.reshape(-1, 1, 19, 3000)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.cell_count, self.input_size)
        _x = None

        for index, cell in enumerate(self.lstm):
            cell_out, _ = cell(x[:, index, :, :], None)
            last_layer_out = cell_out[:, -1, :]
            
            last_layer_out = last_layer_out.unsqueeze(0)
            if _x is None:
                _x = last_layer_out
            else:
                _x = torch.cat((_x, last_layer_out), dim=0)
            

        x = _x.permute(1, 0, 2).unsqueeze(1)

        x = self.cnn_block(x)


        x = x.reshape(self.N, -1)

        x = self.fcn(x)
        x = self.results(x)

        return x

In [7]:


# Set the directory path containing the folders
data_dir = r"C:\Users\admin\Desktop\MNE_Data"
dataset = 0
labels = []
dataset = []
# Get a list of all folders in the directory
folders = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]
# Loop through each folder
for folder in folders:
    folder_path = os.path.join(data_dir, folder)
    files = [f for f in os.listdir(folder_path) if f.endswith('.fif')]
    print(folder)

    for file in files:
        file_path = os.path.join(folder_path, file)

        raw = mne.read_epochs(file_path).get_data(picks='eeg');
        raw = Normalize(raw)

        dataset.append(raw)

        if folder == 'FOCAL':
            labels.append(np.zeros((raw.shape[0], 1)))

        elif folder == 'IGE':
            labels.append(np.ones((raw.shape[0], 1)))

        elif folder == 'PNES':
            labels.append(np.ones((raw.shape[0], 1)) * 2)

        elif folder == 'TLE':
            labels.append(np.ones((raw.shape[0], 1)) * 3)
        
dataset = np.concatenate(dataset, axis=0)
labels = np.array(labels)
labels = np.concatenate(labels).ravel().reshape(-1, 1)
        

FOCAL
IGE
PNES
TLE


In [8]:
dataset = torch.from_numpy(dataset).float()
labels = torch.from_numpy(labels).long()
dataset, X_unseen, initial_labels, y_unseen = train_test_split(dataset, labels, test_size=0.25, random_state=42, stratify=labels)
#del dataset

In [9]:
model = net(T = 3000, C = 19, input_size = 3000, hidden_size = 100, num_layers=1, spatial_num= 500, dropout=0.2, pool=1).to(device)

criterion = nn.CrossEntropyLoss(weight = torch.Tensor([5.3125, 1.8333, 3.5417, 6.6667]).cuda())

optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.Adagrad(model.parameters(), lr=0.001)


scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.1)
epochs = 80
batch_size = 1024
kf = StratifiedKFold(n_splits=4, shuffle=True)

In [ ]:
for fold, (train_ids, test_ids) in enumerate(kf.split(dataset, initial_labels)):
    print(f'FOLD {fold + 1}')
    X_train = dataset[train_ids].cuda()
    y_train = initial_labels[train_ids].cuda()
    X_test = dataset[test_ids].cuda()
    y_test = initial_labels[test_ids].cuda()
    
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

    for epoch in trange(epochs):
        # Set the model to training mode
        model.train()
        # Initialize the running loss
        running_loss = 0.0
        # Loop over the train batches
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs)
            # Compute the loss
            labels = F.one_hot(labels.squeeze(), num_classes=4).float()
            loss = criterion(outputs, labels)
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            # Update the running loss
            running_loss += loss.item()
            # Print the statistics every 200 batches
            torch.cuda.empty_cache()

        test()

    

In [11]:
torch.cuda.empty_cache()

In [12]:
yy_unseen = F.one_hot(y_unseen.squeeze(), num_classes=4).cpu()

In [13]:
ground_truth_labels = np.argmax(yy_unseen, axis=1)
#logits = model(X_unseen.cuda())


In [14]:
#model_output_labels = np.argmax(model(X_unseen.cuda()), axis=1)
#confusion_mat = confusion_matrix(ground_truth_labels, model_output_labels)